In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import gc, math

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

# Preprocessing Test

In [2]:
test_df = pd.read_csv('../raw_data/test.csv', parse_dates=['timestamp'])

In [3]:
X = {'building_id':[104],'meter':[0],'timestamp':['2017-06-02 12:00:00']}

In [4]:
def preprocess(X):

    weather_preproc = pd.read_csv('../raw_data/weather_preproc.csv', parse_dates=['timestamp'])

    building_preproc = pd.read_csv("../raw_data/building_preproc.csv")


    X = pd.DataFrame.from_dict(X)
    
    X['timestamp'] = pd.to_datetime(X.timestamp)
    row = X.merge(building_preproc.iloc[:,1:], on='building_id', how='left')

    row = row.merge(weather_preproc.iloc[:,1:], on=['site_id', 'timestamp'], how='left')

    beaufort = [(0, 0, 0.3), (1, 0.3, 1.6), (2, 1.6, 3.4), (3, 3.4, 5.5), (4, 5.5, 8), (5, 8, 10.8), (6, 10.8, 13.9), 
              (7, 13.9, 17.2), (8, 17.2, 20.8), (9, 20.8, 24.5), (10, 24.5, 28.5), (11, 28.5, 33), (12, 33, 200)]

    for item in beaufort:
        row.loc[(row['wind_speed']>=item[1]) & (row['wind_speed']<item[2]), 'beaufort_scale'] = item[0]

    row["timestamp"] = pd.to_datetime(row["timestamp"])

    def transform(df):
        df['hour'] = np.uint8(df['timestamp'].dt.hour)
        df['day'] = np.uint8(df['timestamp'].dt.day)
        df['weekday'] = np.uint8(df['timestamp'].dt.weekday)
        df['month'] = np.uint8(df['timestamp'].dt.month)
        df['year'] = np.uint8(df['timestamp'].dt.year-1900)

        df['square_feet'] = np.log(df['square_feet'])

        return df

    row = transform(row)

    def encode_cyclic_feature(df, col, max_vals):
        df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_vals)
    #     df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_vals)
        del df[col]
        return df

    dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
    us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

    row['is_holiday'] = (row['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
    row['is_holiday'] = (row['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

    row.loc[(row['weekday']) == 5 | (row['weekday'] == 6) , 'is_holiday'] = 1

    row = row.drop(['timestamp'], axis=1)

    row = encode_cyclic_feature(row, 'weekday', 7)
    row = encode_cyclic_feature(row, 'hour', 24)
    row = encode_cyclic_feature(row, 'day', 31)
    row = encode_cyclic_feature(row, 'month', 12)

    row = row.loc[:,['site_id', 'building_id', 'primary_use', 'meter', 'wind_direction',
           'is_holiday', 'square_feet', 'year_built', 'air_temperature',
           'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'floor_count',
           'beaufort_scale', 'weekday_sin', 'day_sin', 'hour_sin', 'month_sin']]
    return row

In [5]:
X = preprocess(X)

In [6]:
def pred(X):
    model1 = lgb.Booster(model_file='model1.txt')
    model2 = lgb.Booster(model_file='model2.txt')
    y_pred = (model1.predict(X)+model2.predict(X))/2
    
    #to_return = {'meter_reading':float(y_pred)}

    return y_pred

In [7]:
pred(X)

array([5.78088669])

## Monthly Prediction

In [8]:
dates

NameError: name 'dates' is not defined

In [9]:
X1 = {'building_id':[104],'meter':[0],'timestamp':['2017-06-02 12:00:00']}
X2 = {'building_id':[104],'meter':[0],'timestamp':['2017-06-02 13:00:00']}
X3 = {'building_id':[104],'meter':[0],'timestamp':['2017-06-02 14:00:00']}

In [10]:
X1 = preprocess(X1)
X2 = preprocess(X2)
X3 = preprocess(X3)

In [11]:
pd.concat([X1,X2,X3])

,site_id,building_id,primary_use,meter,wind_direction,is_holiday,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,floor_count,beaufort_scale,weekday_sin,day_sin,hour_sin,month_sin
0,0,104,6,0,120.0,0,10.721724,2003.0,26.7,7.000000,23.9,-1.0,4.0,1.0,-0.433884,0.394356,1.224647e-16,1.224647e-16
0,0,104,6,0,240.0,0,10.721724,2003.0,28.3,6.666667,23.3,0.0,4.0,3.0,-0.433884,0.394356,-2.588190e-01,1.224647e-16
0,0,104,6,0,295.0,0,10.721724,2003.0,29.4,6.333333,23.3,0.0,4.0,2.0,-0.433884,0.394356,-5.000000e-01,1.224647e-16


In [115]:
bu_id = 100
meter = 0
year = 2017
month = 1

In [116]:
first_timestamp = "2017-01-01 0:00"

last_timestamp = "2017-01-31 23:00"

In [117]:
dates = pd.date_range("2017-01-01 0:00", "2017-01-31 23:00", freq="1h")

dates=pd.DataFrame(dates)

dates

,0
0,2017-01-01 00:00:00
1,2017-01-01 01:00:00
2,2017-01-01 02:00:00
3,2017-01-01 03:00:00
4,2017-01-01 04:00:00
...,...
739,2017-01-31 19:00:00
740,2017-01-31 20:00:00
741,2017-01-31 21:00:00
742,2017-01-31 22:00:00


In [118]:
dates['building_id'] = 100
dates['meter'] = 0
dates

,0,building_id,meter
0,2017-01-01 00:00:00,100,0
1,2017-01-01 01:00:00,100,0
2,2017-01-01 02:00:00,100,0
3,2017-01-01 03:00:00,100,0
4,2017-01-01 04:00:00,100,0
...,...,...,...
739,2017-01-31 19:00:00,100,0
740,2017-01-31 20:00:00,100,0
741,2017-01-31 21:00:00,100,0
742,2017-01-31 22:00:00,100,0


In [119]:
dates = dates.iloc[:,[1,2,0]]

In [120]:
dates

,building_id,meter,0
0,100,0,2017-01-01 00:00:00
1,100,0,2017-01-01 01:00:00
2,100,0,2017-01-01 02:00:00
3,100,0,2017-01-01 03:00:00
4,100,0,2017-01-01 04:00:00
...,...,...,...
739,100,0,2017-01-31 19:00:00
740,100,0,2017-01-31 20:00:00
741,100,0,2017-01-31 21:00:00
742,100,0,2017-01-31 22:00:00


In [121]:
X = dates.rename(columns={0:'timestamp'})

In [122]:
X

,building_id,meter,timestamp
0,100,0,2017-01-01 00:00:00
1,100,0,2017-01-01 01:00:00
2,100,0,2017-01-01 02:00:00
3,100,0,2017-01-01 03:00:00
4,100,0,2017-01-01 04:00:00
...,...,...,...
739,100,0,2017-01-31 19:00:00
740,100,0,2017-01-31 20:00:00
741,100,0,2017-01-31 21:00:00
742,100,0,2017-01-31 22:00:00


In [123]:
X = preprocess(X)

In [124]:
X

,site_id,building_id,primary_use,meter,wind_direction,is_holiday,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,floor_count,beaufort_scale,weekday_sin,day_sin,hour_sin,month_sin
0,0,100,4,0,140.0,0,10.104631,1968.0,17.2,0.0,13.3,0.0,4.0,2.0,-0.781831,2.012985e-01,0.000000,0.5
1,0,100,4,0,130.0,0,10.104631,1968.0,16.7,2.0,13.3,0.0,4.0,2.0,-0.781831,2.012985e-01,0.258819,0.5
2,0,100,4,0,130.0,0,10.104631,1968.0,15.6,2.0,12.8,0.0,4.0,2.0,-0.781831,2.012985e-01,0.500000,0.5
3,0,100,4,0,150.0,0,10.104631,1968.0,15.0,0.0,12.8,0.0,4.0,1.0,-0.781831,2.012985e-01,0.707107,0.5
4,0,100,4,0,0.0,0,10.104631,1968.0,15.0,2.0,13.3,0.0,4.0,0.0,-0.781831,2.012985e-01,0.866025,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,0,100,4,0,0.0,1,10.104631,1968.0,21.1,0.0,6.1,0.0,4.0,0.0,0.781831,-2.449294e-16,-0.965926,0.5
740,0,100,4,0,150.0,1,10.104631,1968.0,18.3,2.0,7.8,0.0,4.0,2.0,0.781831,-2.449294e-16,-0.866025,0.5
741,0,100,4,0,170.0,1,10.104631,1968.0,16.1,0.0,7.2,0.0,4.0,1.0,0.781831,-2.449294e-16,-0.707107,0.5
742,0,100,4,0,180.0,1,10.104631,1968.0,14.4,0.0,7.8,0.0,4.0,1.0,0.781831,-2.449294e-16,-0.500000,0.5


In [125]:
sum(pred(X))

TypeError: pred() missing 2 required positional arguments: 'meter' and 'dates'

In [127]:
X.iloc[:,6]

0      10.104631
1      10.104631
2      10.104631
3      10.104631
4      10.104631
         ...    
739    10.104631
740    10.104631
741    10.104631
742    10.104631
743    10.104631
Name: square_feet, Length: 744, dtype: float64

In [23]:
X.square_feet /= 2

In [24]:
X.primary_use = 1

In [25]:
pred(X)

array([3.835858  , 3.7971138 , 3.8557426 , 3.87217165, 3.86009864,
       3.87818546, 3.85185261, 3.84336234, 3.87822437, 3.83997115,
       3.84227473, 3.81812358, 3.70247595, 3.75869456, 3.79046098,
       3.80899257, 3.83376487, 3.84651388, 3.83376487, 3.82968856,
       3.80524798, 3.8204728 , 3.79610449, 3.73651375, 3.74155285,
       3.77048368, 3.77583478, 3.8010854 , 3.79326731, 3.81791273,
       3.83736347, 3.8616719 , 3.81968278, 3.83635079, 3.80957219,
       3.72447633, 3.7468753 , 3.7909232 , 3.79965416, 3.8314438 ,
       3.82584731, 3.8303211 , 3.84852012, 3.82890616, 3.79495063,
       3.77428303, 3.75893454, 3.77015751, 3.77559434, 3.77854718,
       3.76802112, 3.75349413, 3.73208359, 3.77111039, 3.7707679 ,
       3.76157342, 3.7708068 , 3.76972396, 3.78711169, 3.73683228,
       3.794152  , 3.79982807, 3.82018752, 3.82877262, 3.83951051,
       3.8709185 , 3.83951051, 3.82462036, 3.84809719, 3.82440247,
       3.8037365 , 3.80131989, 3.80254255, 3.77420079, 3.77211

In [80]:
building_preproc = pd.read_csv("../raw_data/building_preproc.csv")
building_preproc

,Unnamed: 0,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,0,7432,2008.0,4.0
1,1,0,1,0,2720,2004.0,4.0
2,2,0,2,0,5376,1991.0,4.0
3,3,0,3,0,23685,2002.0,4.0
4,4,0,4,0,116607,1975.0,4.0
...,...,...,...,...,...,...,...
1444,1444,15,1444,1,19619,1914.0,4.0
1445,1445,15,1445,0,4298,1968.0,4.0
1446,1446,15,1446,1,11265,1997.0,4.0
1447,1447,15,1447,4,29775,2001.0,4.0


In [135]:
building_info = building_preproc.loc[(building_preproc['building_id'] == 2)]

In [136]:
building_info

,Unnamed: 0,site_id,building_id,primary_use,square_feet,year_built,floor_count
2,2,0,2,0,5376,1991.0,4.0


In [137]:
building_info_reset = building_info.drop(['Unnamed: 0'], axis = 1)

In [138]:
building_info_reset

,site_id,building_id,primary_use,square_feet,year_built,floor_count
2,0,2,0,5376,1991.0,4.0


In [105]:
del building_info_reset['index']

KeyError: 'index'

In [148]:
building_info_reset

,site_id,building_id,primary_use,square_feet,year_built,floor_count
2,0,2,0,5376,1991.0,4.0


In [149]:
building_dic = building_info_reset.to_dict(orient = 'records')

In [151]:
building_dic[0]

{'site_id': 0,
 'building_id': 2,
 'primary_use': 0,
 'square_feet': 5376,
 'year_built': 1991.0,
 'floor_count': 4.0}

In [155]:
def info(building_id):
    #X = {'building_id': [int(building_id)],'meter': [int(meter)], 'period': [int(dates)]}
    #df = preprocess(X)
    
    building_preproc = pd.read_csv("../raw_data/building_preproc.csv")

    df = building_preproc.filter(items = [2], axis=0)
    
    to_return =  df
    #df.to_json()

    return to_return

In [156]:
#X = {'building_id':[104],'meter':[0],'timestamp':['2017-06-02 12:00:00']}
info(104)

,Unnamed: 0,site_id,building_id,primary_use,square_feet,year_built,floor_count
2,2,0,2,0,5376,1991.0,4.0


In [ ]:
type()

In [161]:
def info(building_id):
    #X = {'building_id': [int(building_id)],'meter': [int(meter)], 'period': [int(dates)]}
    #df = preprocess(X)

    building_preproc = pd.read_csv("../raw_data/building_preproc.csv")
    building_info = building_preproc.loc[(building_preproc['building_id'] == building_id)]
    building_info_reset = building_info.drop(['Unnamed: 0'], axis = 1)
    building_dic = building_info_reset.to_dict(orient = 'records')

    to_return = building_dic[0]

    return to_return

In [162]:
info(2)

{'site_id': 0,
 'building_id': 2,
 'primary_use': 0,
 'square_feet': 5376,
 'year_built': 1991.0,
 'floor_count': 4.0}

In [170]:
def pred_eff(building_id: int, meter: int, initial_date: str, final_date: str):
    initial_timestamp = initial_date +' '+"0:00"
    final_timestamp = final_date + ' '+ "23:00"

    dates = pd.date_range(initial_timestamp, final_timestamp, freq="1h")
    input_values = pd.DataFrame(dates)
    input_values['building_id'] = building_id
    input_values['meter'] = meter
    input_values = input_values.iloc[:,[1,2,0]]
    print(input_values)
    input_values = input_values.rename({0:'timestamp'}, axis = 1)

    X = preprocess(input_values)
    #X = {'building_id': [int(building_id)],'meter': [int(meter)], 'period': [int(dates)]}

    #X = dates.rename(columns={0:'timestamp'})
    #X = preprocess(X)

    model1 = lgb.Booster(model_file='../raw_data/model1.txt')
    model2 = lgb.Booster(model_file='../raw_data/model2.txt')
    y_pred = ((model1.predict(X)+model2.predict(X))/2)/X.iloc[:,6]

    y_pred = pd.DataFrame(y_pred,columns=['Cons. (kwh)'],index = input_values.timestamp )

    to_return = y_pred

    return to_return

In [171]:
pred_eff(104, 1, '17-04-2022', '18-04-2022')

    building_id  meter                   0
0           104      1 2022-04-17 00:00:00
1           104      1 2022-04-17 01:00:00
2           104      1 2022-04-17 02:00:00
3           104      1 2022-04-17 03:00:00
4           104      1 2022-04-17 04:00:00
5           104      1 2022-04-17 05:00:00
6           104      1 2022-04-17 06:00:00
7           104      1 2022-04-17 07:00:00
8           104      1 2022-04-17 08:00:00
9           104      1 2022-04-17 09:00:00
10          104      1 2022-04-17 10:00:00
11          104      1 2022-04-17 11:00:00
12          104      1 2022-04-17 12:00:00
13          104      1 2022-04-17 13:00:00
14          104      1 2022-04-17 14:00:00
15          104      1 2022-04-17 15:00:00
16          104      1 2022-04-17 16:00:00
17          104      1 2022-04-17 17:00:00
18          104      1 2022-04-17 18:00:00
19          104      1 2022-04-17 19:00:00
20          104      1 2022-04-17 20:00:00
21          104      1 2022-04-17 21:00:00
22         

,Cons. (kwh)
timestamp,
2022-04-17 00:00:00,NaN
2022-04-17 01:00:00,NaN
2022-04-17 02:00:00,NaN
2022-04-17 03:00:00,NaN
2022-04-17 04:00:00,NaN
2022-04-17 05:00:00,NaN
2022-04-17 06:00:00,NaN
2022-04-17 07:00:00,NaN
2022-04-17 08:00:00,NaN
